# Next Steps


1. Rewrite optModel and optDataset module
2. Write a training loop
3. Define regret and track regret/runtime
4. Plot

## Test optModel

In [118]:
import cvxpy as cp
import numpy as np
import warnings
import sys
from IPython.core.interactiveshell import InteractiveShell
import torch
import torch.optim as optim
from torch import nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, TensorDataset, random_split

np.random.seed(42)
torch.manual_seed(42)

warnings.filterwarnings("ignore")
InteractiveShell.ast_node_interactivity = "all"
sys.path.insert(1,'/Users/dennis/Documents/myGit/main/myUtils')

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)


cpu


In [119]:
class optModel:
    def __init__(self, x, r, c, Q, alpha):
        self.alpha = alpha
        self.Q = Q
        self.r = r
        self.c = c
        self.x = x
        self.num_data, self.num_items, self.num_features = x.shape

    def setObj(self, r, c):
        if self.alpha == 1:
            self.objective = cp.sum(cp.log(cp.multiply(r, self.d)))
        else:
            self.objective = cp.sum(cp.power(cp.multiply(r, self.d), 1 - self.alpha)) / (1 - self.alpha)
        
        self.constraints = [
            self.d >= 0,
            cp.sum(cp.multiply(c, self.d)) <= self.Q
        ]
        self.problem = cp.Problem(cp.Maximize(self.objective), self.constraints)

    def solve(self, closed=False):
        opt_sol = []
        opt_val = []
        if closed:
            return self.solveC()

        for i in range(self.num_data):
            self.d = cp.Variable(self.num_items)
            self.setObj(self.r[i], self.c[i])
            self.problem.solve(abstol=1e-9, reltol=1e-9, feastol=1e-9)
            opt_sol.append(self.d.value.reshape(1, self.num_items))
            opt_val.append(self.problem.value)

        opt_sol = np.concatenate(opt_sol)
        return opt_sol, opt_val

    def solveC(self):
        if self.alpha == 1:
            return "Work in progress"
        
        opt_sols_c = []
        opt_vals_c = []
        for i in range(self.num_data):
            S = np.sum(self.c[i] ** (1 - 1 / self.alpha) * self.r[i] ** (-1 + 1 / self.alpha))
            opt_sol_c = (self.c[i] ** (-1 / self.alpha) * self.r[i] ** (-1 + 1 / self.alpha) * self.Q) / S
            opt_val_c = np.sum((self.r[i] * opt_sol_c) ** (1 - self.alpha)) / (1 - self.alpha)
            opt_sols_c.append(opt_sol_c)
            opt_vals_c.append(opt_val_c)
        
        opt_sols_c = np.array(opt_sols_c)
        return opt_sols_c, opt_vals_c

## 生成数据 

In [120]:
def genData(num_data, num_features, num_items, seed=42, Q=100, dim=1, deg=1, noise_width=0.5, epsilon=0.1):
    rnd = np.random.RandomState(seed)
    n = num_data
    p = num_features
    m = num_items
    
    x = rnd.normal(0, 1, (n, m, p))
    B = rnd.binomial(1, 0.5, (m, p))

    c = np.zeros((n, m))
    for i in range(n):
        for j in range(m):
            values = (np.dot(B[j], x[i, j].reshape(p, 1)).T / np.sqrt(p) + 3) ** deg + 1
            values *= 5
            values /= 3.5 ** deg
            epislon = rnd.uniform(1 - noise_width, 1 + noise_width, 1)
            values *= epislon
            values = np.ceil(values)
            c[i, j] = values

    c = c.astype(np.float64)
    r = rnd.normal(0, 1, (n, m))
    r = 1 / (1 + np.exp(-r))

    return x, r, c, Q

In [121]:
# Test optModel with synthetic data
x, r, c, Q = genData(100, 4, 3)
alpha = 0.5

# Create an instance of the optModel class
optmodel = optModel(x, r, c, Q, alpha)

# Solve the optimization problem
opt_sol, opt_val = optmodel.solve()

print("Optimal solution:", opt_sol[0])
print("Objective value:", opt_val[0])

opt_sol_c, opt_val_c = optmodel.solve(closed=True)

print("Optimal solution (closed form):", opt_sol_c[0])
print("Objective value (closed form):", opt_val_c[0])

# Are they the same?
np.allclose(opt_sol, opt_sol_c, atol=1e-4, rtol=1e-4)
np.allclose(opt_val, opt_val_c)

Optimal solution: [ 3.70912192 13.14701444  3.06401269]
Objective value: 12.051416435784253
Optimal solution (closed form): [ 3.70911821 13.14703314  3.06400571]
Objective value (closed form): 12.051416436072254


True

True

## 测试optDataset

In [122]:
class optDataset(Dataset):
    def __init__(self, x, r, c, Q, alpha, closed=False):
        self.closed = closed
        self.x = x
        self.r = r
        self.c = c
        self.Q = Q
        self.alpha = alpha
        self.num_data, self.num_items, self.num_features = x.shape

        self._solve_optimization_problems()

    def _solve_optimization_problems(self):
        optmodel = optModel(self.x, self.r, self.c, self.Q, self.alpha)
        self.opt_sols, self.opt_vals = optmodel.solve()
        self.opt_sols_c, self.opt_vals_c = optmodel.solve(closed=True)
        self.opt_vals = np.array(self.opt_vals)
        self.opt_vals_c = np.array(self.opt_vals_c)

    def __len__(self):
        return len(self.r)

    def __getitem__(self, idx):
        if self.closed:
            return (
                self.x[idx],
                self.r[idx],
                self.c[idx],
                self.opt_sols[idx],
                self.opt_vals[idx],
                self.opt_sols_c[idx],
                self.opt_vals_c[idx],
            )
        else:
            return (
                self.x[idx],
                self.r[idx],
                self.c[idx],
                self.opt_sols[idx],
                self.opt_vals[idx],
            )



In [123]:
# Test optDatasetRd
dataset = optDataset(x, r, c, Q, alpha)
dataset[0]

(array([[ 0.49671415, -0.1382643 ,  0.64768854,  1.52302986],
        [-0.23415337, -0.23413696,  1.57921282,  0.76743473],
        [-0.46947439,  0.54256004, -0.46341769, -0.46572975]]),
 array([0.65990733, 0.76377236, 0.54513221]),
 array([7., 4., 7.]),
 array([ 3.70912192, 13.14701444,  3.06401269]),
 12.051416435784253)

In [128]:
num_items = x.shape[1]
num_features = x.shape[2]
print("Number of items:", num_items)
print("Number of features:", num_features)

class LogisticRegressionModel(nn.Module):
    def __init__(self, num_items, num_features):
        super(LogisticRegressionModel, self).__init__()
        self.num_items = num_items
        self.num_features = num_features
        self.linears = nn.ModuleList([nn.Linear(num_features, 1) for _ in range(num_items)])

    def forward(self, x):
        outputs = []
        for i in range(self.num_items):
            outputs.append(torch.sigmoid(self.linears[i](x[:, i, :])))
        return torch.cat(outputs, dim=1)


Number of items: 3
Number of features: 4


In [129]:
# Define the model
model = LogisticRegressionModel(num_items, num_features).to(device)

# Train-test split
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

batch_size = 32
loader_train = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
loader_test = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

# Training loop with custom loss (regret)
num_epochs = 50
for epoch in range(num_epochs):
    model.train()
    train_loss = 0.0
    for x_batch, r_batch, c_batch, opt_sols_batch, opt_vals_batch in loader_train:
        x_batch = x_batch.float().to(device)
        r_batch = r_batch.float().to(device)
        c_batch = c_batch.float().to(device)
        opt_vals_batch = opt_vals_batch.float().to(device)

        optimizer.zero_grad()
        pred_r = model(x_batch)
        
        regret_loss = calRegret(optModel, x_batch.cpu().numpy(), c_batch.cpu().numpy(), pred_r.cpu().detach().numpy(), r_batch.cpu().numpy(), opt_vals_batch.cpu().numpy(), alpha)
        
        loss = criterion(pred_r, r_batch) + regret_loss
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item()

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {train_loss/len(loader_train):.4f}')


LogisticRegressionModel(
  (linears): ModuleList(
    (0): Linear(in_features=4, out_features=1, bias=True)
    (1): Linear(in_features=4, out_features=1, bias=True)
    (2): Linear(in_features=4, out_features=1, bias=True)
  )
)

Epoch [1/50], Loss: 7.9709


LogisticRegressionModel(
  (linears): ModuleList(
    (0): Linear(in_features=4, out_features=1, bias=True)
    (1): Linear(in_features=4, out_features=1, bias=True)
    (2): Linear(in_features=4, out_features=1, bias=True)
  )
)

Epoch [2/50], Loss: 7.5159


LogisticRegressionModel(
  (linears): ModuleList(
    (0): Linear(in_features=4, out_features=1, bias=True)
    (1): Linear(in_features=4, out_features=1, bias=True)
    (2): Linear(in_features=4, out_features=1, bias=True)
  )
)

...

Epoch [48/50], Loss: 5.0785


LogisticRegressionModel(
  (linears): ModuleList(
    (0): Linear(in_features=4, out_features=1, bias=True)
    (1): Linear(in_features=4, out_features=1, bias=True)
    (2): Linear(in_features=4, out_features=1, bias=True)
  )
)

Epoch [49/50], Loss: 5.0780


LogisticRegressionModel(
  (linears): ModuleList(
    (0): Linear(in_features=4, out_features=1, bias=True)
    (1): Linear(in_features=4, out_features=1, bias=True)
    (2): Linear(in_features=4, out_features=1, bias=True)
  )
)

Epoch [50/50], Loss: 5.0831


## Regret

In [114]:
def calRegret(optmodel, x, true_c, pred_r, true_r, true_obj, alpha):
    model = optmodel(x,pred_r, true_c, Q, alpha)
    sol, _ = model.solve()
    val = []
    for i in range(x.shape[1]):
        temp = np.sum((true_r[i] * sol[i]) ** (1 - alpha)) / (1 - alpha)
        val.append(temp)
    val = np.array(val)
    regret_loss = 0
    for i in range(x.shape[1]):
        regret_loss += true_obj[i] - val[i]
    return regret_loss

In [115]:
def regret(predmodel, optmodel, dataloader, closed=False, alpha=0.5):
    """
    A function to evaluate model performance with normalized true regret

    Args:
        predmodel (nn): a regression neural network for cost prediction
        optmodel (optModel): an PyEPO optimization model
        dataloader (DataLoader): Torch dataloader from optDataSet

    Returns:
        float: true regret loss
    """
    # eval
    predmodel.eval()
    loss = 0
    optsum = 0

    if not closed:
    # load data
        for data in dataloader:
            x,r,c,opt_sol,opt_obj,_,_ = data
            # cuda
            x,r,c,opt_sol,opt_obj = x.cuda(),r.cuda(),c.cuda(),opt_sol.cuda(),opt_obj.cuda()
            # predict
            with torch.no_grad():
                pred_r = predmodel(x).to('cpu').detach().numpy()
            # solve
            for j in range(pred_r.shape[0]):
                loss += calRegret(optmodel, c[j], pred_r[j], r[j].to("cpu").detach().numpy(), opt_obj[j].item(), alpha)

                optsum += abs(opt_obj[j].item())
    # turn back to train mode
    predmodel.train()

    # normalize
    return loss / (optsum+1e-7)